<b>Data mining project - 2020/21</b><br>
<b>Author</b>: [Alexandra Bradan](https://github.com/alexandrabradan)<br>
<b>Python version</b>: 3.x<br>
<b>Last update: 07/01/2021<b>

In [1]:
%matplotlib inline

# general libraries
import sys
import math
import operator
import itertools
import collections
from pylab import MaxNLocator
from collections import Counter
from collections import defaultdict
from IPython.display import Image

# pandas libraries
import pandas as pd
from pandas import DataFrame
from pandas.testing import assert_frame_equal

# visualisation libraries
import seaborn as sns
import matplotlib.pyplot as plt

# numpy libraries
import numpy as np
from numpy import std
from numpy import mean
from numpy import arange
from numpy import unique
from numpy import percentile

# scipy libraries
import scipy.stats as stats
from scipy.stats import kstest
from scipy.stats import normaltest

# sklearn libraries
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.experimental import enable_iterative_imputer  # explicitly require this experimental feature
from sklearn.impute import IterativeImputer

from sklearn import tree
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as imbPipeline
from imblearn.pipeline import make_pipeline as imbmake_pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import RepeatedStratifiedKFold 
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import accuracy_score, f1_score, fbeta_score, recall_score, precision_score, classification_report, roc_auc_score 

In [2]:
data_directory = "../../../data/"
plot_directory = "../../../plots/DataUnderstanding/"
TR_file = data_directory + "Train_HR_Employee_Attrition.csv"
TR_cleaned_file = data_directory + "Numerical_Encoding_Train_HR_Employee_Attrition.csv"
TS_file = data_directory + "Numerical_Encoding_Test_HR_Employee_Attrition.csv"

In [3]:
df_cleaned = pd.read_csv(TR_cleaned_file, sep=",") 
df_ts = pd.read_csv(TS_file, sep=",") 

In [4]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       883 non-null    int64  
 1   Attrition                 883 non-null    int64  
 2   BusinessTravel            883 non-null    int64  
 3   DistanceFromHome          883 non-null    int64  
 4   Education                 883 non-null    int64  
 5   EnvironmentSatisfaction   883 non-null    int64  
 6   Gender                    883 non-null    int64  
 7   JobInvolvement            883 non-null    int64  
 8   JobLevel                  883 non-null    int64  
 9   JobRole                   883 non-null    int64  
 10  JobSatisfaction           883 non-null    int64  
 11  MonthlyIncome             883 non-null    int64  
 12  NumCompaniesWorked        883 non-null    int64  
 13  OverTime                  883 non-null    int64  
 14  PercentSal

In [5]:
df_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       219 non-null    int64  
 1   Attrition                 219 non-null    int64  
 2   BusinessTravel            219 non-null    int64  
 3   DistanceFromHome          219 non-null    int64  
 4   Education                 219 non-null    int64  
 5   EnvironmentSatisfaction   219 non-null    int64  
 6   Gender                    219 non-null    int64  
 7   JobInvolvement            219 non-null    int64  
 8   JobLevel                  219 non-null    int64  
 9   JobRole                   219 non-null    int64  
 10  JobSatisfaction           219 non-null    int64  
 11  MonthlyIncome             219 non-null    int64  
 12  NumCompaniesWorked        219 non-null    int64  
 13  OverTime                  219 non-null    int64  
 14  PercentSal

<h2> Discretisation approach </h2> 
Approaches to transform continuous variables into discrete ones. This process is also known as <b>binning</b>, with each bin being each interval. Discretization methods fall into 2 categories: 

- supervised: do not use any information, other than the variable distribution, to create the contiguous bins in which the values will be placed;
- unsupervised: typically use target information in order to create bins or intervals.

Since we are dealying with DT it is natural to use a **supervised discretisation method** with them:

<u>Step 1</u>: First it trains a decision tree of limited depth (2, 3 or 4) using the variable we want to discretize to predict the target;

<u>Step 2</u>: The original variable values are then replaced by the probability returned by the tree. The probability is the same for all the observations within a single bin, thus replacing by the probability is equivalent to grouping the observations within the cut-off decided by the decision tree.

**Advantages** :
- The probabilistic predictions returned decision tree are monotonically related to the target.
- The new bins show decreased entropy, this is the observations within each bucket/bin are more similar to themselves than to those of other buckets/bins.
- The tree finds the bins automatically.

**Disadvantages**:
- It may cause over-fitting
- More importantly, some tuning of tree parameters might need to be done to obtain the optimal splits (e.g., depth, the minimum number of samples in one partition, the maximum number of partitions, and a minimum information gain). This it can be time-consuming.

<u>Features to discretize</u>:
- Age
- DistanceFromHome
- YearsAtCompany
- YearsInCurrentRole
- NumCompaniesWorked
- MonthlyIncome
- MonthlyHours

- PercentSalaryHike
- TaxRate

In [6]:
X_train = df_cleaned.copy()
X_test = df_ts.copy()

In [7]:
def print_performed_encoding(column_name, train_encoded):
    column_index = df_cleaned.columns.get_loc(column_name)
    encoding_info = {}
    for enc, i in zip(train_encoded, range(0, len(train_encoded))):
        try:
            tmp_list = encoding_info[str(enc)]
            tmp_list.append(df_cleaned.iloc[i, column_index])
            encoding_info[str(enc)] = tmp_list
        except KeyError:
            encoding_info[str(enc)] = [df_cleaned.iloc[i, column_index]]
            
    for key, value in encoding_info.items():
        min_value = min(value)
        max_value = max(value)
        print(column_name, key, "[%s-%s]" %(min_value, max_value),sep="\t")

In [8]:
def equal_frequency_discretizer(column_name, best_k):
    discretizer = KBinsDiscretizer(n_bins=best_k, encode='ordinal', strategy='quantile').fit(X_train[[column_name]])
    train_encoded = discretizer.transform(X_train[[column_name]]).astype(int)
    test_encoded = discretizer.transform(X_test[[column_name]]).astype(int)
    X_train[column_name] = train_encoded
    X_test[column_name] = test_encoded
    print_performed_encoding(column_name, train_encoded)

<h6>Age </h6>
Build a classification tree using the Age to predict Attrition in order to discretise the age variable

In [9]:
best_k = 6
column_name = "Age"
equal_frequency_discretizer(column_name, best_k)

Age	[5]	[45-60]
Age	[3]	[35-38]
Age	[0]	[18-28]
Age	[2]	[32-34]
Age	[1]	[29-31]
Age	[4]	[39-44]


<h6>DistanceFromHome </h6>
Build a classification tree using the variable to predict Attrition in order to discretise it

In [10]:
best_k = 6
column_name = "DistanceFromHome"
equal_frequency_discretizer(column_name, best_k)

DistanceFromHome	[3]	[7-9]
DistanceFromHome	[0]	[1-1]
DistanceFromHome	[4]	[10-18]
DistanceFromHome	[2]	[3-6]
DistanceFromHome	[5]	[19-29]
DistanceFromHome	[1]	[2-2]


<h6> YearsAtCompany </h6>

In [11]:
best_k = 5
column_name = "YearsAtCompany"
equal_frequency_discretizer(column_name, best_k)

YearsAtCompany	[3]	[7-9]
YearsAtCompany	[0]	[0-1]
YearsAtCompany	[1]	[2-4]
YearsAtCompany	[4]	[10-20]
YearsAtCompany	[2]	[5-6]


<h6> YearsInCurrentRole </h6>

In [12]:
best_k = 4
column_name = "YearsInCurrentRole"
equal_frequency_discretizer(column_name, best_k)

YearsInCurrentRole	[3]	[7-16]
YearsInCurrentRole	[2]	[3-6]
YearsInCurrentRole	[1]	[2-2]
YearsInCurrentRole	[0]	[0-1]


<h6> NumCompaniesWorked </h6>

In [13]:
best_k = 2
column_name = "NumCompaniesWorked"
equal_frequency_discretizer(column_name, best_k)

NumCompaniesWorked	[1]	[2-9]
NumCompaniesWorked	[0]	[0-1]


<h6> MonthlyIncome </h6>

In [14]:
best_k = 11
column_name = "MonthlyIncome"
equal_frequency_discretizer(column_name, best_k)

MonthlyIncome	[8]	[8318-10400]
MonthlyIncome	[5]	[4728-5410]
MonthlyIncome	[2]	[2739-3306]
MonthlyIncome	[6]	[5429-6384]
MonthlyIncome	[0]	[1009-2311]
MonthlyIncome	[3]	[3312-4060]
MonthlyIncome	[1]	[2318-2725]
MonthlyIncome	[9]	[10422-14961]
MonthlyIncome	[10]	[15232-26997]
MonthlyIncome	[7]	[6385-8268]
MonthlyIncome	[4]	[4071-4724]


<h6> MonthlyHours </h6>

In [15]:
best_k = 3
column_name = "MonthlyHours"
equal_frequency_discretizer(column_name, best_k)

MonthlyHours	[0]	[26.04347826086957-163.67073170731706]
MonthlyHours	[2]	[271.78205128205127-590.9767441860465]
MonthlyHours	[1]	[163.70666666666668-271.75]


<h6> PercentSalaryHike </h6>

In [16]:
best_k = 4
column_name = "PercentSalaryHike"
equal_frequency_discretizer(column_name, best_k)

PercentSalaryHike	[2]	[14-17]
PercentSalaryHike	[3]	[18-25]
PercentSalaryHike	[1]	[12-13]
PercentSalaryHike	[0]	[11-11]


<h6> TaxRate </h6>

In [17]:
best_k = 4
column_name = "TaxRate"
equal_frequency_discretizer(column_name, best_k)

TaxRate	[1]	[0.13020833333333334-0.5271877403703923]
TaxRate	[2]	[0.528359484706026-0.7622001654259719]
TaxRate	[3]	[0.7629205375665193-0.9513959334891722]
TaxRate	[0]	[0.0-0.12925247468426446]


<h6> OverallSatisfaction </h6>

In [18]:
best_k = 6
column_name = "OverallSatisfaction"
equal_frequency_discretizer(column_name, best_k)

OverallSatisfaction	[2]	[2.6-2.6]
OverallSatisfaction	[3]	[2.8-2.8]
OverallSatisfaction	[5]	[3.2-4.0]
OverallSatisfaction	[1]	[2.4-2.4]
OverallSatisfaction	[4]	[3.0-3.0]
OverallSatisfaction	[0]	[1.2-2.2]


<h2> Discretize variables and save them on new file </h2>

In [19]:
discrete_variables = ["DistanceFromHome", "YearsAtCompany", "YearsInCurrentRole", "NumCompaniesWorked",
                      "MonthlyIncome", "PercentSalaryHike", "TaxRate", "Age", "MonthlyHours", "OverallSatisfaction"]
for var in discrete_variables:
    print("train", var, sorted(X_train[var].unique()))
    print("test", var, sorted(X_test[var].unique()))

train DistanceFromHome [0, 1, 2, 3, 4, 5]
test DistanceFromHome [0, 1, 2, 3, 4, 5]
train YearsAtCompany [0, 1, 2, 3, 4]
test YearsAtCompany [0, 1, 2, 3, 4]
train YearsInCurrentRole [0, 1, 2, 3]
test YearsInCurrentRole [0, 1, 2, 3]
train NumCompaniesWorked [0, 1]
test NumCompaniesWorked [0, 1]
train MonthlyIncome [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
test MonthlyIncome [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
train PercentSalaryHike [0, 1, 2, 3]
test PercentSalaryHike [0, 1, 2, 3]
train TaxRate [0, 1, 2, 3]
test TaxRate [0, 1, 2, 3]
train Age [0, 1, 2, 3, 4, 5]
test Age [0, 1, 2, 3, 4, 5]
train MonthlyHours [0, 1, 2]
test MonthlyHours [0, 1, 2]
train OverallSatisfaction [0, 1, 2, 3, 4, 5]
test OverallSatisfaction [0, 1, 2, 3, 4, 5]


In [20]:
print(X_train.shape)
print(X_test.shape)

(883, 24)
(219, 24)


In [21]:
df1 = X_train.copy()
df2 = X_test.copy()

In [22]:
df1.to_csv(data_directory + "Discretized_Quantile_Numerical_Encoding_Train_HR_Employee_Attrition.csv", index=False, header=True)
df2.to_csv(data_directory + "Discretized_Quantile_Numerical_Encoding_Test_HR_Employee_Attrition.csv", index=False, header=True)

In [23]:
df_discretized = pd.read_csv(data_directory + "Discretized_Quantile_Numerical_Encoding_Train_HR_Employee_Attrition.csv", sep=",") 
df_discretized.shape

(883, 24)

In [24]:
df_discretized = pd.read_csv(data_directory + "Discretized_Quantile_Numerical_Encoding_Test_HR_Employee_Attrition.csv", sep=",") 
df_discretized.shape

(219, 24)